## Do zrobienia:

- Usunięcie zbędnych kolumn - takich jak np. informacje o dacie ceremonii otwarcia poszczególnych igrzysk.
- Utworzenie kolumny sport type - wiąże się to z zaklasyfikowaniem poszczególnych sportów do odpowiednio zdefiniowanych typów (np. sporty drużynowe, wodne itp.)
- Utworzenie indeksów zastępczych przy tabelach sportowców i igrzysk (igrzyska mogą być identyfikowane przez połączenie kolum year + season, warto więc wprowadzić numeryczny indeks, który ułatwi odwoływanie się z tabeli faktowej).
- Ujednolicenie sposobu zapisu nazw drużyn. -> Nie rozumiem.
- Ujednolicenie nazw lokalizacji (część rekordów opisuje już nieistniejące kraje, jak imperialna Japonia czy nazistowskie Niemcy.
- Dopasowanie NOC do państw (niektóre kody mogą być przestarzałe). -> Nie widzę problemu. Widocznie w roku igrzysk dane państwo nazywało się inaczej niż teraz.

In [1]:
import numpy as np
import pandas as pd
import copy
from datetime import date

In [2]:
df_athlete_events = pd.read_csv('new_athlete_events.csv', delimiter=';')
df_athlete_events.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [3]:
df_player_scores = pd.read_csv('player_scores.csv', delimiter=',')
df_player_scores.head(3)

,player,score
0,1,4
1,2,4
2,3,5


In [4]:
df_noc_regions = pd.read_csv('noc_regions.csv', delimiter=',')
df_noc_regions.head(3)

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN


In [5]:
df_cities = pd.read_csv('list-host-cities-olympic-943j.csv', delimiter=',')
df_cities.drop([62,63], axis=0, inplace=True)
df_cities

,City,Country,Continent,Summer (Olympiad),Winter,Year,Opening Ceremony,Closing Ceremony
0,Athens,Greece,Europe,I,-,1896.0,April 6,April 15
1,Paris,France,Europe,II,-,1900.0,May 14,October 28
2,St. Louis,United States,North America,III,-,1904.0,July 1,November 23
3,London,United Kingdom,Europe,IV,-,1908.0,April 27,October 31
4,Stockholm,Sweden,Europe,V,-,1912.0,May 5,July 22
...,...,...,...,...,...,...,...,...
57,Beijing,China,Asia,-,XXIV,2022.0,February 4,February 20
58,Paris,France,Europe,XXXIII,-,2024.0,August 2,August 18
59,TBD,TBD,NaN,-,XXV,2026.0,(TBD),NaN
60,Los Angeles,United States,North America,XXXIV,-,2028.0,July 21,August 6


In [6]:
df_games_data = pd.read_csv('games_data.csv', delimiter=',')
df_games_data.head(3)

,games,costs,athletes,events
0,1896 Summer,NaN,241,43
1,1900 Summer,NaN,997,95
2,1904 Summer,NaN,651,95


- Filtrowanie - wybranie tylko potrzebnych do hurtowni atrybutów oraz ograniczenie obserwacji do lat 2000-2016.

In [7]:
df_athlete_events = df_athlete_events[(df_athlete_events.Year >= 2000) & (df_athlete_events.Year <= 2015)]
df_athlete_events.reset_index(drop=True, inplace=True)

In [8]:
df_cities = df_cities[(df_cities.Year >= 2000) & (df_cities.Year <= 2015)]
df_cities.reset_index(drop=True, inplace=True)

## Utworzenie tabelki `Locations`

In [9]:
Locations = df_cities.drop_duplicates(subset=['Continent', 'Country', 'City'])[['Continent', 'Country', 'City']]
Locations.columns = ['continent', 'country', 'city']
Locations.insert(loc=0, column='location_id', value=range(len(Locations)))
Locations

,location_id,continent,country,city
0,0,Oceania,Australia,Sydney
1,1,North America,United States,Salt Lake City
2,2,Europe,Greece,Athens
3,3,Europe,Italy,Turin
4,4,Asia,China,Beijing
5,5,North America,Canada,Vancouver
6,6,Europe,United Kingdom,London
7,7,Europe,Russia,Sochi


## Utworzenie tabelki `Teams`
- Zamiana pustych wartości parametru team country na nazwę kraju.

In [10]:
Teams = df_athlete_events.drop_duplicates(subset=['Team', 'NOC'])[['Team', 'Team', 'NOC']]
Teams.reset_index(inplace=True, drop=True)
Teams.columns = ['team_name', 'team_country', 'NOC']
Teams.insert(loc=0, column='team_id', value=range(len(Teams)))
Teams

,team_id,team_name,team_country,NOC
0,0,China,China,CHN
1,1,Finland,Finland,FIN
2,2,Norway,Norway,NOR
3,3,Netherlands,Netherlands,NED
4,4,France,France,FRA
...,...,...,...,...
323,323,Salinero,Salinero,NED
324,324,Satchmo,Satchmo,GER
325,325,Solos Carex,Solos Carex,SWE
326,326,Dow Jones,Dow Jones,JPN


- Czyszczenie - wypełnienie pustych elementów w tabeli zawodnika wartością domyślną.

In [11]:
df_athlete_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71570 entries, 0 to 71569
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      71570 non-null  int64  
 1   Name    71570 non-null  object 
 2   Sex     71570 non-null  object 
 3   Age     71567 non-null  float64
 4   Height  71035 non-null  float64
 5   Weight  70583 non-null  float64
 6   Team    71570 non-null  object 
 7   NOC     71570 non-null  object 
 8   Games   71570 non-null  object 
 9   Year    71570 non-null  int64  
 10  Season  71570 non-null  object 
 11  City    71570 non-null  object 
 12  Sport   71570 non-null  object 
 13  Event   71570 non-null  object 
 14  Medal   10115 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.2+ MB


Puste wartości w kolumnach: `Age`, `Height`, `Weight`, `Medal`.

In [12]:
mean_age = int(np.mean(df_athlete_events.Age))
mean_height = int(np.mean(df_athlete_events.Height))
mean_weight = int(np.mean(df_athlete_events.Weight))

In [13]:
df_athlete_events.Age.fillna(mean_age, inplace=True)
df_athlete_events.Height.fillna(mean_height, inplace=True)
df_athlete_events.Weight.fillna(mean_weight, inplace=True)

- Zamienić wartości NULL w uzyskanych medalach na informację o braku medalu.

In [14]:
df_athlete_events.Medal.fillna('lack', inplace=True)

In [15]:
df_athlete_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71570 entries, 0 to 71569
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      71570 non-null  int64  
 1   Name    71570 non-null  object 
 2   Sex     71570 non-null  object 
 3   Age     71570 non-null  float64
 4   Height  71570 non-null  float64
 5   Weight  71570 non-null  float64
 6   Team    71570 non-null  object 
 7   NOC     71570 non-null  object 
 8   Games   71570 non-null  object 
 9   Year    71570 non-null  int64  
 10  Season  71570 non-null  object 
 11  City    71570 non-null  object 
 12  Sport   71570 non-null  object 
 13  Event   71570 non-null  object 
 14  Medal   71570 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.2+ MB


Brak pustych wartości.

- Wyliczenie roku urodzenia zawodnika na podstawie wieku zawodnika i roku odbywania się igrzysk.


In [16]:
df_athlete_events['birth_year'] = df_athlete_events.Year-df_athlete_events.Age

## Utworzenie tabelki `Athletes`
- utworzenie indeksów zastępczych

In [17]:
Athletes = df_athlete_events.drop_duplicates(subset=['ID'])[['ID','Name','Sex','birth_year','Height','Weight']]
Athletes.columns = ['athlete_id','full_name','sex','birth_year','height','weight']
Athletes = Athletes.merge(df_player_scores, how='left', left_on='athlete_id', right_on='player')
del Athletes['player']

Athletes.insert(loc=0, column='sid', value=range(len(Athletes)))
Athletes.insert(loc=8, column='date_valid_from', value=date.today())
Athletes.insert(loc=9, column='date_valid_to', value=date.max)
Athletes.insert(loc=10, column='status', value=1)

Athletes.head(3)

,sid,athlete_id,full_name,sex,birth_year,height,weight,score,date_valid_from,date_valid_to,status
0,0,2,A Lamusi,M,1989.0,170.0,60.0,4,2022-06-08,9999-12-31,1
1,1,9,Antti Sami Aalto,M,1976.0,186.0,96.0,4,2022-06-08,9999-12-31,1
2,2,12,Jyri Tapani Aalto,M,1969.0,172.0,70.0,1,2022-06-08,9999-12-31,1


- Utworzenie kolumny medal score i przypisanie poszczególnym medalom odpowiedniej punktacji (opisanej w sekcji Opis kluczowych miar i atrybutów w modelu.

In [18]:
medal_val_dict = {'Gold':5, 'Silver':3, 'Bronze':1, 'lack':0}

In [19]:
df_athlete_events['medal_score'] = df_athlete_events.Medal.replace(medal_val_dict)
df_athlete_events.head(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,birth_year,medal_score
0,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,lack,1989.0,0
1,9,Antti Sami Aalto,M,26.0,186.0,96.0,Finland,FIN,2002 Winter,2002,Winter,Salt Lake City,Ice Hockey,Ice Hockey Men's Ice Hockey,lack,1976.0,0
2,12,Jyri Tapani Aalto,M,31.0,172.0,70.0,Finland,FIN,2000 Summer,2000,Summer,Sydney,Badminton,Badminton Men's Singles,lack,1969.0,0
3,13,Minna Maarit Aalto,F,34.0,159.0,55.5,Finland,FIN,2000 Summer,2000,Summer,Sydney,Sailing,Sailing Women's Windsurfer,lack,1966.0,0
4,16,Juhamatti Tapio Aaltonen,M,28.0,184.0,85.0,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze,1986.0,1


- Pominięcie danych o igrzyskach, które nie odbyły się.

In [20]:
df_cities = df_cities[df_cities['Opening Ceremony'].str.contains('Cancelled')==False]
df_cities

,City,Country,Continent,Summer (Olympiad),Winter,Year,Opening Ceremony,Closing Ceremony
0,Sydney,Australia,Oceania,XXVII,-,2000.0,September 15,October 1
1,Salt Lake City,United States,North America,-,XIX,2002.0,February 8,February 24
2,Athens,Greece,Europe,XXVIII,-,2004.0,August 13,August 29
3,Turin,Italy,Europe,-,XX,2006.0,February 10,February 26
4,Beijing,China,Asia,XXIX,-,2008.0,August 8,August 24
5,Vancouver,Canada,North America,-,XXI,2010.0,February 12,February 28
6,London,United Kingdom,Europe,XXX,-,2012.0,July 27,August 12
7,Sochi,Russia,Europe,-,XXII,2014.0,February 7,February 23


## Utworzenie tabelki `Sports`

In [21]:
np.matrix(np.unique(df_athlete_events.Sport))

matrix([['Alpine Skiing', 'Archery', 'Athletics', 'Badminton',
         'Baseball', 'Basketball', 'Beach Volleyball', 'Biathlon',
         'Bobsleigh', 'Boxing', 'Canoeing', 'Cross Country Skiing',
         'Curling', 'Cycling', 'Diving', 'Equestrianism', 'Fencing',
         'Figure Skating', 'Football', 'Freestyle Skiing', 'Gymnastics',
         'Handball', 'Hockey', 'Ice Hockey', 'Judo', 'Luge',
         'Modern Pentathlon', 'Nordic Combined', 'Rhythmic Gymnastics',
         'Rowing', 'Sailing', 'Shooting', 'Short Track Speed Skating',
         'Skeleton', 'Ski Jumping', 'Snowboarding', 'Softball',
         'Speed Skating', 'Swimming', 'Synchronized Swimming',
         'Table Tennis', 'Taekwondo', 'Tennis', 'Trampolining',
         'Triathlon', 'Volleyball', 'Water Polo', 'Weightlifting',
         'Wrestling']], dtype=object)

Podział na sporty drużynowe/indywidualne.

In [22]:
sports_types_dict = {
    'team_sport' : ['Baseball', 'Basketball', 'Beach Volleyball',
         'Bobsleigh', 'Cross Country Skiing',
         'Curling', 'Football', 'Handball', 'Hockey', 'Ice Hockey', 'Nordic Combined', 
         'Rowing', 'Sailing', 'Shooting', 'Short Track Speed Skating',
         'Skeleton', 'Ski Jumping', 'Snowboarding', 'Softball',
         'Speed Skating', 'Swimming', 'Synchronized Swimming',
         'Table Tennis', 'Taekwondo', 'Tennis', 'Trampolining',
         'Triathlon', 'Volleyball', 'Water Polo', 'Weightlifting',
         'Wrestling']
}

Nieskończone. Nie znam się. Może lepszy będzie podział: drużynowe/ w parach/indywidualne. Niektóre sporty mają podkategorie, w których są i dyscypliny drużynowe i indywidualne, np. Gymnastic.

In [23]:
Sports = df_athlete_events.drop_duplicates(subset=['Sport','Event'])[['Sport','Event']]
Sports.reset_index(inplace=True, drop=True)
Sports.columns = ['sport_name', 'event_name']
Sports.insert(loc=0, column='event_id', value=range(len(Sports)))

Sports

,event_id,sport_name,event_name
0,0,Judo,Judo Men's Extra-Lightweight
1,1,Ice Hockey,Ice Hockey Men's Ice Hockey
2,2,Badminton,Badminton Men's Singles
3,3,Sailing,Sailing Women's Windsurfer
4,4,Athletics,Athletics Men's Shot Put
...,...,...,...
430,430,Trampolining,Trampolining Men's Individual
431,431,Weightlifting,Weightlifting Women's Flyweight
432,432,Cycling,Cycling Women's Omnium
433,433,Cycling,Cycling Women's Team Sprint


In [94]:
len(np.unique(Sports.event_name)) #unikalne

435

- Połączenie przechowywanych oddzielnie dnia i miesiąca oraz roku odbycia się ceremonii otwarcia igrzysk w jedną datę w dla wymiaru daty.

In [24]:
import datetime

In [26]:
df_cities = df_cities.astype({'Year':int})

In [27]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
for i in range(len(df_cities)):
    df_cities.Year[i] = int(df_cities.Year[i])
    df_cities['Opening Ceremony'][i] = datetime.datetime(
        df_cities.Year[i],
        int(datetime.datetime.strptime(df_cities['Opening Ceremony'][i] , '%B %d').strftime('%m')),
        int(datetime.datetime.strptime(df_cities['Opening Ceremony'][i] , '%B %d').strftime('%d'))
    ).strftime('%Y-%m-%d')
df_cities

,City,Country,Continent,Summer (Olympiad),Winter,Year,Opening Ceremony,Closing Ceremony
0,Sydney,Australia,Oceania,XXVII,-,2000,2000-09-15,October 1
1,Salt Lake City,United States,North America,-,XIX,2002,2002-02-08,February 24
2,Athens,Greece,Europe,XXVIII,-,2004,2004-08-13,August 29
3,Turin,Italy,Europe,-,XX,2006,2006-02-10,February 26
4,Beijing,China,Asia,XXIX,-,2008,2008-08-08,August 24
5,Vancouver,Canada,North America,-,XXI,2010,2010-02-12,February 28
6,London,United Kingdom,Europe,XXX,-,2012,2012-07-27,August 12
7,Sochi,Russia,Europe,-,XXII,2014,2014-02-07,February 23


## Utworzenie tabelki `Dates`

In [75]:
Dates = df_cities[['Opening Ceremony','Winter','Year']]
Dates['season'] = Dates['Winter'].apply(lambda x: 'Summer' if x == '-' else 'Winter')
del Dates['Winter']
Dates.columns = ['date','year','season']
Dates.insert(loc=0, column='date_id', value=np.nan)
for i in range(len(Dates)):
    Dates['date_id'][i] = str(str(Dates.year[i]) +' '+ Dates.season[i])
Dates.insert(loc=3, column='month', value=df_cities['Opening Ceremony'].apply(lambda x: int(datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%m'))))
Dates.insert(loc=4, column='day', value=df_cities['Opening Ceremony'].apply(lambda x: int(datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%d'))))

Dates

,date_id,date,year,month,day,season
0,2000 Summer,2000-09-15,2000,9,15,Summer
1,2002 Winter,2002-02-08,2002,2,8,Winter
2,2004 Summer,2004-08-13,2004,8,13,Summer
3,2006 Winter,2006-02-10,2006,2,10,Winter
4,2008 Summer,2008-08-08,2008,8,8,Summer
5,2010 Winter,2010-02-12,2010,2,12,Winter
6,2012 Summer,2012-07-27,2012,7,27,Summer
7,2014 Winter,2014-02-07,2014,2,7,Winter


## Utworzenie tabelki `Olympic games`

In [76]:
df_games_data.head()

,games,costs,athletes,events
0,1896 Summer,NaN,241,43
1,1900 Summer,NaN,997,95
2,1904 Summer,NaN,651,95
3,1906 Summer,NaN,854,78
4,1908 Summer,0.012,2008,110


In [86]:
Olympic_games = copy.deepcopy(df_games_data)
Olympic_games.columns = ['game_id', 'cost', 'athlete_number', 'events_number']
mean_cost = np.mean(Olympic_games.cost)
Olympic_games.cost.fillna(mean_cost, inplace=True)
Olympic_games.insert(loc=1, column='date_id', value=copy.deepcopy(Olympic_games.game_id))

Olympic_games.head()

,game_id,date_id,cost,athlete_number,events_number
0,1896 Summer,1896 Summer,3.664839,241,43
1,1900 Summer,1900 Summer,3.664839,997,95
2,1904 Summer,1904 Summer,3.664839,651,95
3,1906 Summer,1906 Summer,3.664839,854,78
4,1908 Summer,1908 Summer,0.012000,2008,110


Komentarze:
- ograniczenie tabelek ze względu na rok z kolumną postaci '1900 Summer'? (ograniczenie tabel wymiarowych)

## Utworzenie tabelki `Olympic event fact`

In [89]:
df_athlete_events.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,birth_year,medal_score
0,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,lack,1989.0,0
1,9,Antti Sami Aalto,M,26.0,186.0,96.0,Finland,FIN,2002 Winter,2002,Winter,Salt Lake City,Ice Hockey,Ice Hockey Men's Ice Hockey,lack,1976.0,0


In [100]:
Olympic_event_fact = copy.deepcopy(df_athlete_events)[['Medal','ID','NOC','Games','Games','City','Event','medal_score']]
Olympic_event_fact.columns = ['medal_name','athlete_id','NOC','game_id','date_id','City','event_name','medal_score']
Olympic_event_fact.insert(loc=0, column='medal_id', value=range(len(Olympic_event_fact)))

Olympic_event_fact.head(3)

,medal_id,medal_name,athlete_id,NOC,game_id,date_id,City,event_name,medal_score
0,0,lack,2,CHN,2012 Summer,2012 Summer,London,Judo Men's Extra-Lightweight,0
1,1,lack,9,FIN,2002 Winter,2002 Winter,Salt Lake City,Ice Hockey Men's Ice Hockey,0
2,2,lack,12,FIN,2000 Summer,2000 Summer,Sydney,Badminton Men's Singles,0
